## 추석 전일('22.09.09) 12시(정오) 고속도로 서울TG 출발 -> 부산 TG 도착 소요시간 예측

- 기준: 한국도로공사 '도시 간 소요시간(서울-주요도시)' 제공 추석전일('22.09.09) 12시 정오 고속도로 서울-부산 이동시간 최근접 소요시간 예측자
- 응모 양식:
    - 예측 소요시간(txt): 제출하기 태 내 '시:분 소요' 기입
    - 분석 데이터 결과(.ipynb): 실행하여 결과가 포함된 파일
    - 분석 설명서(.pptx): 가설 및 데이터 설명 PPT 1장(자유양식)


In [1]:
import numpy as np
import pandas as pd

import requests, json

import warnings
warnings.filterwarnings('ignore')

In [2]:
# user_key = ''

### 1. 도시간 이동 소요 시간(2016년 이후)

In [3]:
def getIntercity(year, page):
    url = 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime'
    params = {'key': user_key,
              'type': 'json',
              'iYear': str(year),    # 기준년도
              'iStUnitCode': '101',  # 서울TG 영업소 코드
              'iEdUnitCode': '140',  # 부산TG 영업소 코드
              'numOfRows': '50',     # 출력 건수(최대 99)
              'pageNo': str(page)    # 출력 페이지번호
             }
    response = requests.get(url, params)
    return response.json()['intercityLeadTimeLists']

In [4]:
cols = ['stdYear', 'sphlDfttNm', 'sphlDfttScopTypeNm', 'stdHour', 'trtm']
intercity = pd.DataFrame(columns=cols)
i = 0

for y in range(2016, 2023):
    p = 1
    while True:
        data = getIntercity(y, p)
        i = len(data)
        
        if i == 0:
            break
        else:
            print(f'{y}년 {p}번째 데이터 수집 완료')
            df = pd.DataFrame(data)[cols]
            intercity = pd.concat([intercity, df], axis=0)
            p += 1

2016년 1번째 데이터 수집 완료
2016년 2번째 데이터 수집 완료
2017년 1번째 데이터 수집 완료
2017년 2번째 데이터 수집 완료
2017년 3번째 데이터 수집 완료
2017년 4번째 데이터 수집 완료
2018년 1번째 데이터 수집 완료
2018년 2번째 데이터 수집 완료
2018년 3번째 데이터 수집 완료
2018년 4번째 데이터 수집 완료
2018년 5번째 데이터 수집 완료
2018년 6번째 데이터 수집 완료
2018년 7번째 데이터 수집 완료
2018년 8번째 데이터 수집 완료
2018년 9번째 데이터 수집 완료
2019년 1번째 데이터 수집 완료
2019년 2번째 데이터 수집 완료
2019년 3번째 데이터 수집 완료
2019년 4번째 데이터 수집 완료
2019년 5번째 데이터 수집 완료
2019년 6번째 데이터 수집 완료
2019년 7번째 데이터 수집 완료
2019년 8번째 데이터 수집 완료
2020년 1번째 데이터 수집 완료
2020년 2번째 데이터 수집 완료
2020년 3번째 데이터 수집 완료
2020년 4번째 데이터 수집 완료
2020년 5번째 데이터 수집 완료
2020년 6번째 데이터 수집 완료
2020년 7번째 데이터 수집 완료
2020년 8번째 데이터 수집 완료
2020년 9번째 데이터 수집 완료
2021년 1번째 데이터 수집 완료
2021년 2번째 데이터 수집 완료
2021년 3번째 데이터 수집 완료
2021년 4번째 데이터 수집 완료
2021년 5번째 데이터 수집 완료
2021년 6번째 데이터 수집 완료
2021년 7번째 데이터 수집 완료
2021년 8번째 데이터 수집 완료
2021년 9번째 데이터 수집 완료
2022년 1번째 데이터 수집 완료
2022년 2번째 데이터 수집 완료
2022년 3번째 데이터 수집 완료
2022년 4번째 데이터 수집 완료
2022년 5번째 데이터 수집 완료
2022년 6번째 데이터 수집 완료


In [5]:
intercity.columns = ['기준년도', '특수일', '특송기간', '기준시', '통행시간(분)']

In [6]:
intercity.head()

,기준년도,특수일,특송기간,기준시,통행시간(분)
0,2016,연말연시,D,00,325.377666666666666666666666
1,2016,연말연시,D,01,321.183000000000000000000000
2,2016,연말연시,D,02,315.688666666666666666666666
3,2016,연말연시,D,03,309.322000000000000000000000
4,2016,연말연시,D,04,299.949333333333333333333333


In [7]:
intercity.to_csv('./rawdata/intercity_raw.csv', index=False)

### 2. 시간대 별 교통량

In [8]:
def getTraffic(year):
    url = 'http://data.ex.co.kr/openapi/specialAnal/trafficFlowByTime'
    params = {'key': user_key,
              'type': 'json',
              'iStdYear': str(year),    # 기준년도
             }
    response = requests.get(url, params)
    return response.json()['trafficFlowByTimeLists']

In [9]:
cols = ['stdYear', 'sphlDfttNm', 'sphlDfttScopTypeNm', 'stdHour', 'trfl']
traffic = pd.DataFrame(columns=cols)

for y in range(2016, 2023):
        data = getTraffic(y)
        print(f'{y}년 데이터 수집 완료')
        df = pd.DataFrame(data)[cols]
        traffic = pd.concat([traffic, df], axis=0)

2016년 데이터 수집 완료
2017년 데이터 수집 완료
2018년 데이터 수집 완료
2019년 데이터 수집 완료
2020년 데이터 수집 완료
2021년 데이터 수집 완료
2022년 데이터 수집 완료


In [10]:
traffic.columns = ['기준년도', '특수일', '특송기간', '기준시', '교통량(대)']

In [11]:
traffic.head()

,기준년도,특수일,특송기간,기준시,교통량(대)
0,2016년,추석,D-2,11,61030
1,2016년,설날,D,19,269059
2,2016년,설날,D-2,14,242951
3,2016년,설날,D-1,14,247527
4,2016년,추석,D,21,63238


In [12]:
traffic.to_csv('./rawdata/traffic_raw.csv', index=False)

### 3. 휴게소별 날씨 정보

경부고속도로 휴게소 목록
- 서울만남, 기흥, 안성, 망향, 천안호두, 옥산, 죽암, 옥천, 금강, 황건, 추풍령, 김천, 칠곡, 평사, 건천, 경주, 통도사

In [13]:
def getWeather(date):
    url = 'http://data.ex.co.kr/openapi/restinfo/restWeatherList'
    params = {'key': user_key,
              'type': 'json',
              'sdate': str(date), # %Y%m%
              'stdHour': '13'
             }
    response = requests.get(url, params)
    return response.json()['list']

In [14]:
days = [20190912, 20200930, 20210920] # 2019 - 2021년의 추석 전일
cols = ['sdate', 'unitName', 'routeName', 'rainfallValue', 'rainfallValue']
weather = pd.DataFrame(columns=cols)

for day in days:
        data = getWeather(day)
        print(f'{day} 데이터 수집 완료')
        df = pd.DataFrame(data)[cols]
        weather = pd.concat([weather, df], axis=0)

20190912 데이터 수집 완료
20200930 데이터 수집 완료
20210920 데이터 수집 완료


In [15]:
weather.columns = ['날짜', '휴게소명', '도로명', '일강수값', '강수강도값']

In [16]:
weather.head()

,날짜,휴게소명,도로명,일강수값,강수강도값
0,20190912,죽전휴게소,경부선,0.000000,0.000000
1,20190912,안성휴게소(서울),경부선,0.000000,0.000000
2,20190912,옥산휴게소(서울),경부선,0.000000,0.000000
3,20190912,천안(삼)휴게소(서울),경부선,1.100000,1.100000
4,20190912,죽암휴게소(서울),경부선,6.600000,6.600000


In [17]:
weather.to_csv('./rawdata/weather_raw.csv', index=False)